In [6]:
from transformers import BertModel, BertTokenizerFast
import torch
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from transformers import BertModel, BertTokenizerFast
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils.multiclass import unique_labels
import os

In [10]:
# Correct file path
data_path = "/kaggle/input/data-set/translated_distinct_records_final_version.csv"

# Ensure the dataset exists
if not os.path.exists(data_path):
    raise FileNotFoundError(f"Dataset not found at {data_path}. Ensure it's uploaded and linked to your kernel.")

# Load the dataset
data = pd.read_csv(data_path)

# Apply filter (update the condition based on your needs)
# Example: Filter rows where the 'intent' column matches Siri intents
filter_condition = data['intent'].isin([
    'calendar_set', 'calendar_query', 'weather_query', 'play_music', 'play_audiobook',
    'play_radio', 'email_query', 'sendemail', 'alarm_query', 'alarm_set',
    'datetime_query', 'social_post', 'qa_factoid', 'general_query', 'news_query'
])
filtered_data = data[filter_condition]

# Stop execution if no rows match the filter
if filtered_data.empty:
    raise ValueError("Filtered dataset is empty. No matching rows found with the given condition.")

# Display filtered data (optional)
print(filtered_data.head())

      id                                         transcript         intent  \
0   9024                                              event   calendar_set   
1   7874  i need an event three days from now scheduled ...   calendar_set   
4   3033                         what's the current weather  weather_query   
8  16742  check my email for new emails during the last ...    email_query   
9   5172               please wake me up at seven thirty am      alarm_set   

                                      slots:  \
0                                         []   
1  ['date=three days from now' 'person=amy']   
4                                         []   
8                         ['time=last hour']   
9                   ['time=seven thirty am']   

                                                 cam  \
0                                     calendar_set()   
1  calendar_set(date=three days from now, person=...   
4                                    weather_query()   
8                 

In [11]:
data = filtered_data

In [12]:
le = LabelEncoder()
data['intent_num'] = le.fit_transform(data['intent'])
map_intent = dict(zip(le.transform(le.classes_), le.classes_))
map_intent

{0: 'alarm_query',
 1: 'alarm_set',
 2: 'calendar_query',
 3: 'calendar_set',
 4: 'datetime_query',
 5: 'email_query',
 6: 'news_query',
 7: 'play_audiobook',
 8: 'play_music',
 9: 'play_radio',
 10: 'qa_factoid',
 11: 'sendemail',
 12: 'social_post',
 13: 'weather_query'}

In [13]:
input = list(data['transcript_hebrew'])
input

['אני צריך שתקבעי לי אירוע',
 'אני צריך שתקבעי לי אירוע מתוכנן בעוד שלושה ימים עם איימי',
 'אני רוצה לדעת מה מזג האוויר הנוכחי',
 'עיין בדוא"ל שלי להודעות דוא"ל חדשות במהלך השעה האחרונה',
 'תשימי לי שעון מעורר לשבע וחצי בבוקר',
 'את יכולה להראות לי את השעונים המתוכננים?',
 'אני רוצה שתזכירי לי את הפגישה הבאה עם חברה שלי בשעה שמונה אחר הצהריים ביום ראשון הבא',
 'אני רוצה שתנגני את רשימת ההשמעה טקטיקות מאפליקציית מוזיקה',
 'תבדקי לי מה מזג האוויר',
 "את יכולה לבדוק מתי אני אמור להיפגש פעם הבאה עם ג'ו?",
 'אנא הזכירי לי על פגישת עסקים באחת עשרה בבוקר',
 'אנא תציג לי את השעון המעורר שקבעת לפגישה של יום שלישי',
 'האם אני צריך ללבוש בגדים חמים היום?',
 'תאונת רכבת',
 'האם יש לי תזכורות צפויות',
 'פייסבוק משיקה תג שנוי במחלוקת כדי לפצח חדשות מזויפות',
 'הורד אוצר מילים מסוים',
 'האם תוכל בבקשה ליצור אירוע חדש בלוח השנה שלי',
 'תראה לי את כל האימיילים האחרונים מאנה',
 'הפעל את ספר השמע',
 'ספר לי את הפגישות שלי ביום רביעי בבוקר',
 'קבע שעון מעורר ליום חמישי בשעה שבע אחר הצהריים',
 'הוסף פגישה 

In [14]:
model_name = 'avichr/heBERT'
model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizerFast.from_pretrained(model_name)

#Tokenize and pad the texts
encoded_inputs = tokenizer(input, padding=True,
                            truncation=True,
                            return_tensors='pt')


config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at avichr/heBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/299k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [15]:
with torch.no_grad():
    outputs = model(**encoded_inputs)


In [17]:
# the embedding of the first sentnce with alephbert
outputs[0]

tensor([[[ 1.1576e+00, -4.9337e-01,  5.9105e-01,  ...,  7.1954e-01,
          -1.1960e-02,  3.0632e-01],
         [-5.4613e-02, -3.1928e+00,  8.6231e-01,  ..., -9.0891e-01,
           1.8557e+00,  3.1841e-01],
         [ 1.6476e+00, -1.1507e+00, -2.2124e-01,  ..., -3.9281e-01,
          -8.1048e-01, -1.2460e-01],
         ...,
         [ 4.7266e-01, -1.2965e+00,  3.1110e-01,  ...,  7.2309e-02,
           1.7743e-01,  1.2064e+00],
         [ 4.7273e-01, -1.2958e+00,  3.1156e-01,  ...,  7.3412e-02,
           1.7717e-01,  1.2071e+00],
         [ 4.7258e-01, -1.2970e+00,  3.1069e-01,  ...,  7.1276e-02,
           1.7777e-01,  1.2059e+00]],

        [[ 3.5086e-01, -8.3193e-01,  4.9931e-01,  ...,  8.5360e-01,
           3.6598e-01,  6.2427e-01],
         [-4.7202e-01, -3.4577e+00,  1.0737e+00,  ..., -6.3815e-01,
           1.4689e+00,  7.8641e-01],
         [ 1.5811e+00, -1.2184e+00, -1.8798e-01,  ..., -4.6423e-01,
          -7.4259e-01,  2.4470e-01],
         ...,
         [ 1.1812e-01, -1

In [19]:
token_embeddings = outputs.last_hidden_state

In [20]:
sentence_embeddings = []
for embedding in token_embeddings:
  sentence_embeddings.append(embedding.flatten().numpy())

In [21]:
sentence_embeddings = np.array(sentence_embeddings)

In [23]:
labels = list(data['intent_num'])

In [24]:
X_train, X_test, y_train, y_test = train_test_split(sentence_embeddings, labels, test_size=0.2, random_state=42)

1 - RandomForestClassifier

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import numpy as np


clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)


y_pred = clf.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")


unique_labels_in_test = np.unique(y_test)

print(classification_report(
    y_test,
    y_pred,
    labels=unique_labels_in_test,
    target_names=[le.classes_[i] for i in unique_labels_in_test]
))


Accuracy: 78.54%
                precision    recall  f1-score   support

   alarm_query       0.90      0.33      0.49        27
     alarm_set       0.87      0.63      0.73        41
calendar_query       0.70      0.70      0.70       118
  calendar_set       0.70      0.94      0.80       156
datetime_query       0.96      0.77      0.86        62
   email_query       0.88      0.91      0.89        88
    news_query       0.81      0.82      0.82        96
play_audiobook       1.00      0.11      0.20        37
    play_music       0.73      0.92      0.81       129
    play_radio       0.84      0.48      0.61        56
    qa_factoid       0.73      0.85      0.79        98
   social_post       0.90      0.73      0.81        52
 weather_query       0.90      0.88      0.89       112

      accuracy                           0.79      1072
     macro avg       0.84      0.70      0.72      1072
  weighted avg       0.81      0.79      0.77      1072



2 - logistic regression

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

lr_clf = LogisticRegression(max_iter=1000, random_state=42)
lr_clf.fit(X_train, y_train)


y_pred_lr = lr_clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_lr)
print(f"Accuracy: {accuracy * 100:.2f}%")


unique_labels_in_test = np.unique(y_test)
target_names = [le.classes_[i] for i in unique_labels_in_test]


print(classification_report(
    y_test,
    y_pred_lr,
    labels=unique_labels_in_test,
    target_names=target_names
))


Accuracy: 86.29%
                precision    recall  f1-score   support

   alarm_query       0.91      0.78      0.84        27
     alarm_set       0.80      0.90      0.85        41
calendar_query       0.82      0.79      0.81       118
  calendar_set       0.86      0.92      0.89       156
datetime_query       0.90      0.85      0.88        62
   email_query       0.92      0.94      0.93        88
    news_query       0.85      0.88      0.86        96
play_audiobook       0.79      0.62      0.70        37
    play_music       0.86      0.84      0.85       129
    play_radio       0.82      0.73      0.77        56
    qa_factoid       0.81      0.87      0.84        98
   social_post       0.92      0.85      0.88        52
 weather_query       0.93      0.96      0.95       112

      accuracy                           0.86      1072
     macro avg       0.86      0.84      0.85      1072
  weighted avg       0.86      0.86      0.86      1072



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


3 -- SVM

In [28]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import numpy as np


svm_clf = SVC(kernel='linear', random_state=42)  # Use 'linear' kernel for high-dimensional data
svm_clf.fit(X_train, y_train)


y_pred_svm = svm_clf.predict(X_test)


accuracy = accuracy_score(y_test, y_pred_svm)
print(f"Accuracy: {accuracy * 100:.2f}%")


unique_labels_in_test = np.unique(y_test)
target_names = [le.classes_[i] for i in unique_labels_in_test]


print(classification_report(
    y_test,
    y_pred_svm,
    labels=unique_labels_in_test,
    target_names=target_names
))


Accuracy: 86.19%
                precision    recall  f1-score   support

   alarm_query       0.88      0.81      0.85        27
     alarm_set       0.84      0.88      0.86        41
calendar_query       0.81      0.81      0.81       118
  calendar_set       0.86      0.92      0.89       156
datetime_query       0.91      0.82      0.86        62
   email_query       0.95      0.95      0.95        88
    news_query       0.83      0.85      0.84        96
play_audiobook       0.83      0.68      0.75        37
    play_music       0.86      0.88      0.87       129
    play_radio       0.78      0.68      0.72        56
    qa_factoid       0.81      0.86      0.83        98
   social_post       0.90      0.85      0.87        52
 weather_query       0.94      0.94      0.94       112

      accuracy                           0.86      1072
     macro avg       0.86      0.84      0.85      1072
  weighted avg       0.86      0.86      0.86      1072



transfer learning with fine-tuning using a pre-trained BERT model (hebert)

with early  stop

In [30]:
from transformers import BertModel, BertTokenizerFast, AdamW
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils.multiclass import unique_labels
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")



le = LabelEncoder()
data['intent_num'] = le.fit_transform(data['intent'])
map_intent = dict(zip(le.transform(le.classes_), le.classes_))


intent_counts = data['intent_num'].value_counts()
print("Class distribution:")
print(intent_counts)

input_texts = list(data['transcript_hebrew'])
model_name = 'avichr/heBERT'
model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizerFast.from_pretrained(model_name)


encoded_inputs = tokenizer(input_texts, padding=True, truncation=True, return_tensors='pt')
labels = torch.tensor(data['intent_num'].values)


train_size = 0.8
num_samples = len(labels)
indices = np.arange(num_samples)
np.random.shuffle(indices)
train_indices = indices[:int(train_size * num_samples)]
test_indices = indices[int(train_size * num_samples):]

train_inputs = {key: val[train_indices] for key, val in encoded_inputs.items()}
test_inputs = {key: val[test_indices] for key, val in encoded_inputs.items()}
train_labels = labels[train_indices]
test_labels = labels[test_indices]

class BertForIntentClassification(torch.nn.Module):
    def __init__(self, bert_model, num_classes):
        super(BertForIntentClassification, self).__init__()
        self.bert = bert_model
        self.classifier = torch.nn.Linear(bert_model.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]  # CLS token
        logits = self.classifier(cls_output)
        return logits

num_classes = len(map_intent)
fine_tuned_model = BertForIntentClassification(model, num_classes)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fine_tuned_model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = AdamW(fine_tuned_model.parameters(), lr=2e-5)

epochs = 5
batch_size = 16

train_data = torch.utils.data.TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], train_labels)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

test_data = torch.utils.data.TensorDataset(test_inputs['input_ids'], test_inputs['attention_mask'], test_labels)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)



Class distribution:
intent_num
3     803
8     620
13    559
2     558
10    534
6     500
5     415
4     342
12    279
9     277
1     182
7     150
0     130
11      7
Name: count, dtype: int64


Some weights of BertModel were not initialized from the model checkpoint at avichr/heBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [31]:
# Training loop with Early Stopping
fine_tuned_model.train()

best_loss = float('inf')
patience = 5
early_stop_counter = 0

for epoch in range(epochs):
    total_loss = 0

    # Training
    fine_tuned_model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = [x.to(device) for x in batch]

        optimizer.zero_grad()
        outputs = fine_tuned_model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {avg_loss:.4f}")

    # Validation
    fine_tuned_model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            outputs = fine_tuned_model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(test_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {avg_val_loss:.4f}")

    # Early stopping
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        early_stop_counter = 0
        # Optionally save the best model
        torch.save(fine_tuned_model.state_dict(), "best_model.pth")
    else:
        early_stop_counter += 1
        print(f"No improvement for {early_stop_counter} epochs. Patience left: {patience - early_stop_counter}")

    if early_stop_counter >= patience:
        print("Early stopping triggered. Training stopped.")
        break

# Load the best model if saved
fine_tuned_model.load_state_dict(torch.load("best_model.pth"))

# Evaluation loop
fine_tuned_model.eval()
predictions, true_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = [x.to(device) for x in batch]
        outputs = fine_tuned_model(input_ids, attention_mask)
        preds = torch.argmax(outputs, dim=1)
        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute metrics
accuracy = accuracy_score(true_labels, predictions)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

present_classes = unique_labels(true_labels, predictions)
present_class_names = [map_intent[i] for i in present_classes]

print("Classification Report:")
print(classification_report(true_labels, predictions, target_names=present_class_names))


Epoch 1/5, Training Loss: 0.7767
Epoch 1/5, Validation Loss: 0.3686
Epoch 2/5, Training Loss: 0.2139
Epoch 2/5, Validation Loss: 0.3678
Epoch 3/5, Training Loss: 0.0912
Epoch 3/5, Validation Loss: 0.4401
No improvement for 1 epochs. Patience left: 4
Epoch 4/5, Training Loss: 0.0478
Epoch 4/5, Validation Loss: 0.4419
No improvement for 2 epochs. Patience left: 3
Epoch 5/5, Training Loss: 0.0260
Epoch 5/5, Validation Loss: 0.4583
No improvement for 3 epochs. Patience left: 2
Test Accuracy: 90.39%
Classification Report:
                precision    recall  f1-score   support

   alarm_query       1.00      0.91      0.95        22
     alarm_set       0.85      0.97      0.91        36
calendar_query       0.86      0.87      0.86       123
  calendar_set       0.89      0.94      0.91       139
datetime_query       1.00      0.89      0.94        76
   email_query       0.94      0.95      0.94        94
    news_query       0.92      0.87      0.89       102
play_audiobook       0.71   

increase num epochs to 30

In [33]:
epochs = 30
batch_size = 16

train_data = torch.utils.data.TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], train_labels)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

test_data = torch.utils.data.TensorDataset(test_inputs['input_ids'], test_inputs['attention_mask'], test_labels)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)


In [34]:
# Training loop with Early Stopping
fine_tuned_model.train()

best_loss = float('inf')
patience = 5
early_stop_counter = 0

for epoch in range(epochs):
    total_loss = 0

    # Training
    fine_tuned_model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = [x.to(device) for x in batch]

        optimizer.zero_grad()
        outputs = fine_tuned_model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {avg_loss:.4f}")

    # Validation
    fine_tuned_model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            outputs = fine_tuned_model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(test_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Validation Loss: {avg_val_loss:.4f}")

    # Early stopping
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        early_stop_counter = 0
        # Optionally save the best model
        torch.save(fine_tuned_model.state_dict(), "best_model.pth")
    else:
        early_stop_counter += 1
        print(f"No improvement for {early_stop_counter} epochs. Patience left: {patience - early_stop_counter}")

    if early_stop_counter >= patience:
        print("Early stopping triggered. Training stopped.")
        break

# Load the best model if saved
fine_tuned_model.load_state_dict(torch.load("best_model.pth"))

# Evaluation loop
fine_tuned_model.eval()
predictions, true_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = [x.to(device) for x in batch]
        outputs = fine_tuned_model(input_ids, attention_mask)
        preds = torch.argmax(outputs, dim=1)
        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Compute metrics
accuracy = accuracy_score(true_labels, predictions)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

present_classes = unique_labels(true_labels, predictions)
present_class_names = [map_intent[i] for i in present_classes]

print("Classification Report:")
print(classification_report(true_labels, predictions, target_names=present_class_names))


Epoch 1/30, Training Loss: 0.1111
Epoch 1/30, Validation Loss: 0.4240
Epoch 2/30, Training Loss: 0.0593
Epoch 2/30, Validation Loss: 0.4853
No improvement for 1 epochs. Patience left: 4
Epoch 3/30, Training Loss: 0.0308
Epoch 3/30, Validation Loss: 0.4965
No improvement for 2 epochs. Patience left: 3
Epoch 4/30, Training Loss: 0.0191
Epoch 4/30, Validation Loss: 0.4995
No improvement for 3 epochs. Patience left: 2
Epoch 5/30, Training Loss: 0.0136
Epoch 5/30, Validation Loss: 0.4918
No improvement for 4 epochs. Patience left: 1
Epoch 6/30, Training Loss: 0.0119
Epoch 6/30, Validation Loss: 0.4841
No improvement for 5 epochs. Patience left: 0
Early stopping triggered. Training stopped.
Test Accuracy: 89.27%
Classification Report:
                precision    recall  f1-score   support

   alarm_query       1.00      0.91      0.95        22
     alarm_set       0.86      0.89      0.88        36
calendar_query       0.88      0.80      0.84       123
  calendar_set       0.83      0.92 